Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Lucas Caetano Lopes Rodrigues"
COLLABORATORS = ""

---

# Lista 09 - Comparando Regressões

# Exercício 01:

Analise o desempenho do kNN e de uma Regressão Linear Regularizada para **pelo menos um** dos conjuntos de dados disponível na [seção de regressão linear múltipla](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html) da página do *Livro Understandable Statistics* de Charles Brase e Corrinne Brase. Para o conjunto de dados que escolheu, execute a regressão linear múltipla para explicar o fator $X1$ dos dados ([ver descrição de um dos conjuntos](http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/frame.html)) a partir dos outros fatores. 

Para a questão, faça as seguintes tarefas:

* Realize treino, validação e teste
* Compare as métricas no teste
* Diferente da lista anterior, reporte o erro quadrado médio no conjunto de teste.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [3]:
# The data (X1, X2, X3, X4, X5, X6, X7) are for each city.
# X1 = total overall reported crime rate per 1 million residents
# X2 = reported violent crime rate per 100,000 residents
# X3 = annual police funding in $/resident
# X4 = % of people 25 years+ with 4 yrs. of high school
# X5 = % of 16 to 19 year-olds not in highschool and not highschool graduates.
# X6 = % of 18 to 24 year-olds in college
# X7 = % of people 25 years+ with at least 4 years of college
# https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/frames/mlr06.html

cnames = {'X1': 'crime rate',
          'X2': 'violent crime rate',
          'X3': 'police funding',
          'X4': '25 years highschool',
          'X5': '16 to 19 not in highschool',
          'X6': '18 to 24 in college',
          'X7': '25 years college'}
df = pd.read_excel('mlr06.xls')
df.rename(columns = cnames, inplace=True)

*** No CODEPAGE record, no encoding_override: will use 'ascii'


In [4]:
# Desejamos prever o crime rate baseado nas outras features
# Para isso, utilizaremos os modelos KNN e Regressão Linear Regularizada
# Primeiro vamos dividir o dataset em treino e testes
y = df['crime rate']
X = df.copy().drop('crime rate', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
# ... e z-normalizar os dados de treino e de teste
scaler = StandardScaler()
Z_train = scaler.fit_transform(X_train)
Z_train = pd.DataFrame(Z_train, index=X_train.index, columns=X_train.columns)

Z_test = scaler.fit_transform(X_test)
Z_test = pd.DataFrame(Z_test, index=X_test.index, columns=X_test.columns)

In [6]:
params = {"n_neighbors": list(range(1,25))}
# folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

knr = KNeighborsRegressor()
knr_model = GridSearchCV(knr, params, cv=5)
knr_model.fit(Z_train, y_train)
knr_y_hat = knr_model.predict(Z_test)

print('Erro quadrado KNR: ', mean_squared_error(y_test, knr_y_hat))
print('R2 score KNR: ', r2_score(y_test, knr_y_hat))

Erro quadrado KNR:  52718.386666666665
R2 score KNR:  0.4238586081859268


In [7]:
# Agora testaremos o modelo de regressão linear
# Utilização do GridSearch para selecionar um número adequado de features
lr = LinearRegression()
params = {"n_features_to_select": [1,2,3,4,5,6]}
rfe = RFE(lr)

lr_model = GridSearchCV(rfe, params, scoring= 'r2', cv = 5, return_train_score=True)
lr_model.fit(Z_train, y_train)
lr_y_hat = lr_model.predict(Z_test)

print('Erro quadrado Linear Regression: ', mean_squared_error(y_test, lr_y_hat))
print('R2 score Linear Regression: ', r2_score(y_test, lr_y_hat))

Erro quadrado Linear Regression:  62945.6328287891
R2 score Linear Regression:  0.31208849891594737


Explique e discuta sobre os resultados encontrados no campo abaixo.

Estranhamente os resultados estão variando substancialmente a cada vez que eu executo o notebook. Não sei dizer se é um problema no dataset que eu escolhi, no método de seleção dos conjuntos KFold ou na implementação dos meus modelos - ou todas as alternativas.

Em algumas execuções o método de regressão KNN é muito melhor que a regressão linear enquanto em outras é exatamente o contrário.